<a href="https://colab.research.google.com/github/bkkaggle/pytorch-CycleGAN-and-pix2pix/blob/master/pix2pix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

In [ ]:
# !git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

In [4]:
import os
os.chdir(r'C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix/')

In [3]:
!pip install -r requirements.txt

# Datasets

Download one of the official datasets with:

-   `bash ./datasets/download_pix2pix_dataset.sh [cityscapes, night2day, edges2handbags, edges2shoes, facades, maps]`

Or use your own dataset by creating the appropriate folders and adding in the images. Follow the instructions [here](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/blob/master/docs/datasets.md#pix2pix-datasets).

### First check if dataset name is same for A-B pix2pix pair: (pix2pix requires same image pair)

In [86]:
import os
import random
import shutil
from glob import glob
import os
import pandas as pd

# Set up the paths for directories A and B
dir_a = r'\\shelter\Kyu\unstain2stain\tiles\registered_tiles\HE'
dir_b = r'\\shelter\Kyu\unstain2stain\tiles\registered_tiles\US'
dir_a = glob(os.path.join(dir_a,'*','*.png'))
dir_b = glob(os.path.join(dir_b,'*','*.png'))
dir_a = [os.path.basename(x) for x in dir_a]
dir_b = [os.path.basename(x) for x in dir_b]
dir_a = [x.split('xy')[1] for x in dir_a] # split on xy to get the image #
dir_b = [x.split('xy')[1] for x in dir_b] # split on xy to get the image #

split_a_images = set(dir_a)
split_b_images = set(dir_b)
only_in_a = split_a_images - split_b_images
only_in_b = split_b_images - split_a_images

print(len(only_in_a))
print(len(only_in_b))

0
0


### Dataset creation for HE (B) and NS (A), filter >70% whitespace images and then randomly shuffle images and move them to train/test/val split local folder:

In [3]:
from glob import glob
import os
from natsort import natsorted
hepath = r'\\shelter\Kyu\unstain2stain\tiles\registered_tiles\HE'
all_he_path = glob(os.path.join(hepath,'*','*.png')) # list of all images
nspath = r'\\shelter\Kyu\unstain2stain\tiles\registered_tiles\US'
all_ns_path = glob(os.path.join(nspath,'*','*.png')) # list of all images
all_he_path = natsorted(all_he_path)
all_ns_path = natsorted(all_ns_path)

In [4]:
# Set the seed for random shuffle
from tqdm import tqdm
import math
import random
import pandas as pd

# filter all_he_path:
comp_src = r'\\shelter\Kyu\unstain2stain\tiles\registered_tiles\HE\composition_per_tile\pkl'
# unstained_path = r'\\shelter\Kyu\unstain2stain\tiles\registrated_tiles\Unstained'
desired_he_path = []
desired_ns_path = []
for idx in tqdm(range(len(all_he_path)),total=len(all_he_path),desc='Images Processed',colour='red'):
    #extract image name:
    hename = all_he_path[idx]
    imgname = hename.split("HE")[1].split("\\")[1]
    tilename = os.path.basename(hename)
    tilename = tilename[:-4]
    pkl_src = os.path.join(comp_src,imgname + ".pkl")
    if not os.path.exists(pkl_src):
        continue #images where compositoin doesn't exist, can't use
    pkl = pd.read_pickle(pkl_src)
    comp_df = pkl[tilename]
    if comp_df["white"] > 70 and not math.isnan(comp_df["white"]): # if >70% whitespace and whitespace is not nan, it is a background white image, so drop.
        continue
    else:
        desired_he_path.append(hename)
        nsname = all_ns_path[idx]
        desired_ns_path.append(nsname)


Images Processed: 100%|██████████| 274647/274647 [9:34:49<00:00,  7.96it/s]   


In [5]:
desired_he_path
print(len(desired_he_path))

79271


In [6]:
desired_ns_path
print(len(desired_ns_path))

79271


### Total 79271 images out 274647

In [7]:
# SAVE THE LIST:
df_src = r'\\shelter\Kyu\unstain2stain\tiles\registered_tiles\HE\composition_per_tile'
he_df = pd.DataFrame(desired_he_path)
he_df.to_excel(os.path.join(df_src,'he_train_image_path.xlsx'))
ns_df = pd.DataFrame(desired_ns_path)
ns_df.to_excel(os.path.join(df_src,'ns_train_image_path.xlsx'))

In [8]:
# free memory:
del(desired_he_path)
del(desired_ns_path)

### Now we have paths to images that we desire, copy those images to local, divide to test/train/val and train model:

In [1]:
import os
import random
import shutil
from glob import glob
import os
import pandas as pd
from tqdm import tqdm
import math
import random
from natsort import natsorted

In [3]:
import pandas as pd
he_src = r"\\shelter\Kyu\unstain2stain\tiles\registered_tiles\HE\composition_per_tile\he_train_image_path.xlsx"
ns_src = r"\\shelter\Kyu\unstain2stain\tiles\registered_tiles\HE\composition_per_tile\ns_train_image_path.xlsx"
he_df = pd.read_excel(he_src)
ns_df = pd.read_excel(ns_src)
all_he_path = list(he_df.iloc[:,1]) # 79271 paths
all_ns_path = list(ns_df.iloc[:,1]) # 79271 paths
all_he_path = natsorted(all_he_path)
all_ns_path = natsorted(all_ns_path)

### Check first if images are the same for all 79271 paths!!

In [4]:
new_he_path = [x.split('HE')[1].split("\\")[1].split("_he")[0] + "_" + x.split('HE')[1].split("\\")[2] for x in all_he_path]
new_ns_path = [x.split('Unstained')[1].split("\\")[1].split("_he")[0] + "_" + x.split('Unstained')[1].split("\\")[2] for x in all_ns_path]

In [11]:
new_he_path[79270]

'OTS_14832_16_17510_33894xy1377.png'

In [12]:
new_ns_path[79270]

'OTS_14832_16_72245_41330xy1377.png'

In [13]:
# load in desired image paths:
he_src = r"\\shelter\Kyu\unstain2stain\tiles\registered_tiles\HE\composition_per_tile\he_train_image_path.xlsx"
ns_src = r"\\shelter\Kyu\unstain2stain\tiles\registered_tiles\HE\composition_per_tile\ns_train_image_path.xlsx"
he_df = pd.read_excel(he_src)
ns_df = pd.read_excel(ns_src)
all_he_path = list(he_df.iloc[:,1]) # 79271 paths
all_ns_path = list(ns_df.iloc[:,1]) # 79271 paths
all_he_path = natsorted(all_he_path)
all_ns_path = natsorted(all_ns_path)
# define dest path (change accordingly)
# train_path = r'C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B\train'
# valid_path = r'C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B\val'
# test_path = r'C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B\test'
he_train_path = r'C:\Users\labadmin\PycharmProjects\wirtzlab_pix2pix\datasets\B\train'
he_valid_path = r'C:\Users\labadmin\PycharmProjects\wirtzlab_pix2pix\datasets\B\val'
he_test_path = r'C:\Users\labadmin\PycharmProjects\wirtzlab_pix2pix\datasets\B\test'

if not os.path.exists(he_train_path):
    os.makedirs(he_train_path)

if not os.path.exists(he_valid_path):
    os.makedirs(he_valid_path)

if not os.path.exists(he_test_path):
    os.makedirs(he_test_path)

ns_train_path = r'C:\Users\labadmin\PycharmProjects\wirtzlab_pix2pix\datasets\A\train'
ns_valid_path = r'C:\Users\labadmin\PycharmProjects\wirtzlab_pix2pix\datasets\A\val'
ns_test_path = r'C:\Users\labadmin\PycharmProjects\wirtzlab_pix2pix\datasets\A\test'

if not os.path.exists(ns_train_path):
    os.makedirs(ns_train_path)

if not os.path.exists(ns_valid_path):
    os.makedirs(ns_valid_path)

if not os.path.exists(ns_test_path):
    os.makedirs(ns_test_path)

In [16]:
# # Example showing, keep running this and you get the same lists
# seed = random.random()
# random.seed(seed)
# all_he_path = list(he_df.iloc[:,1]) # 79271 paths
# all_ns_path = list(ns_df.iloc[:,1]) # 79271 paths
# all_paths = list(zip(all_he_path,all_ns_path))
# random.shuffle(all_paths)
# all_he_path, all_ns_path = zip(*all_paths)
# print(all_he_path[1:10])
# print(all_ns_path[1:10])

('\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\\OTS_14684_9_he\\53350_28774xy4567.png', '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\\OTS_14684_3_he\\36966_61542xy2685.png', '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\\OTS_14684_9_he\\75878_49254xy6545.png', '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\\OTS_14684_2_he\\54374_30822xy4242.png', '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\\OTS_14684_7_he\\19558_18534xy1818.png', '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\\OTS_14684_1_he\\23654_33894xy2057.png', '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\\OTS_14684_7_he\\36966_42086xy3541.png', '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\\OTS_14832_13_he\\89190_35942xy8377.png', '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\HE\\OTS_14684_7_he\\30822_54374xy2953.png')
('\\\\shelter\\Kyu\\unstain2stain\\tiles\\registrated_tiles\\U

In [24]:
#80-10-10 split
train_pct = 0.8
valid_pct = 0.1
test_pct = 0.1

seed = random.random()
random.seed(seed)
all_paths = list(zip(all_he_path,all_ns_path))
random.shuffle(all_paths)
all_he_path, all_ns_path = zip(*all_paths)

he_image_files = [os.path.basename(x) for x in all_he_path] # ex 1124_1124xy0001.png basename of image
he_img_type = [x.split("HE")[1].split("\\")[1] for x in all_he_path] #ex. OTS_14684_3. name of image so we can know what image it's from
ns_image_files = [os.path.basename(x) for x in all_ns_path]
ns_img_type = [x.split("Unstained")[1].split("\\")[1] for x in all_ns_path]

num_files = len(he_image_files)
num_train = int(num_files * train_pct)
num_valid = int(num_files * valid_pct)
num_test = int(num_files * test_pct)

# Copy the image files into the train folder
for i in tqdm(range(num_train),total=num_train, colour='red',desc = 'Files Processed'):
    he_src_path = all_he_path[i]
    he_dst_path = os.path.join(he_train_path, he_img_type[i] + "_" + he_image_files[i])
    shutil.copyfile(he_src_path, he_dst_path)
    ns_src_path = all_ns_path[i]
    ns_dst_path = os.path.join(ns_train_path, ns_img_type[i] + "_" + ns_image_files[i])
    shutil.copyfile(ns_src_path, ns_dst_path)

# Copy the image files into the validation folder
for i in tqdm(range(num_train, num_train+num_valid),total=num_train+num_valid-num_train, colour='red',desc = 'Files Processed'):
    he_src_path = all_he_path[i]
    he_dst_path = os.path.join(he_valid_path, he_img_type[i] + "_" + he_image_files[i])
    shutil.copyfile(he_src_path, he_dst_path)
    ns_src_path = all_ns_path[i]
    ns_dst_path = os.path.join(ns_valid_path, ns_img_type[i] + "_" + ns_image_files[i])
    shutil.copyfile(ns_src_path, ns_dst_path)

# Copy the image files into the test folder
for i in tqdm(range(num_train+num_valid, num_files),total=num_files-(num_train+num_valid),colour='red',desc = 'Files Processed'):
    he_src_path = all_he_path[i]
    he_dst_path = os.path.join(he_test_path, he_img_type[i] + "_" + he_image_files[i])
    shutil.copyfile(he_src_path, he_dst_path)
    ns_src_path = all_ns_path[i]
    ns_dst_path = os.path.join(ns_test_path, ns_img_type[i] + "_" + ns_image_files[i])
    shutil.copyfile(ns_src_path, ns_dst_path)


Files Processed: 100%|██████████| 7928/7928 [10:50<00:00, 12.19it/s]


### Rename images so that it matches the requirements, A and B must have same name.

In [17]:
import os
from tqdm import tqdm
A_src = r'C:\Users\labadmin\PycharmProjects\wirtzlab_pix2pix\datasets\A'
splits = os.listdir(r'C:\Users\labadmin\PycharmProjects\wirtzlab_pix2pix\datasets\A')
B_src = r'C:\Users\labadmin\PycharmProjects\wirtzlab_pix2pix\datasets\B'
A_dirs = [os.path.join(A_src,x) for x in splits]
B_dirs = [os.path.join(B_src,x) for x in splits]

for idx in tqdm(range(len(A_dirs)),desc="Files Processed",colour='red',total = len(A_dirs)):
    dir = A_dirs[idx]
    dirs = os.listdir(dir)
    abs_dirs = [os.path.join(dir,x) for x in dirs]
    for idx in range(len(dirs)):
        directory = dirs[idx]
        split_dirs = directory.split("_")
        abs_dir = abs_dirs[idx]
        img_name = split_dirs[0]+"_"+split_dirs[1]+"_"+split_dirs[2]
        xy = split_dirs[4].split("xy")[1].split(".")[0]
        new_directory = os.path.join(dir,img_name + "_" + xy + ".png")
        os.rename(abs_dir,new_directory)
for idx in tqdm(range(len(B_dirs)),desc="Files Processed",colour='red',total = len(B_dirs)):
    dir = B_dirs[idx]
    dirs = os.listdir(dir)
    abs_dirs = [os.path.join(dir,x) for x in dirs]
    for idx in range(len(dirs)):
        directory = dirs[idx]
        split_dirs = directory.split("_")
        abs_dir = abs_dirs[idx]
        img_name = split_dirs[0]+"_"+split_dirs[1]+"_"+split_dirs[2]
        xy = split_dirs[5].split("xy")[1].split(".")[0]
        new_directory = os.path.join(dir,img_name + "_" + xy + ".png")
        os.rename(abs_dir,new_directory)

Files Processed: 100%|██████████| 3/3 [00:10<00:00,  3.40s/it]


In [18]:
import os
from natsort import natsorted
dir_a = r'C:\Users\labadmin\PycharmProjects\wirtzlab_pix2pix\datasets\A'
dir_b = r'C:\Users\labadmin\PycharmProjects\wirtzlab_pix2pix\datasets\B'

# Loop through each split (train, val, test)
for split in ['train']:
    split_a_path = os.path.join(dir_a, split)
    split_b_path = os.path.join(dir_b, split)

    # Get the set of image file names for each split in directory A
    split_a_images = set(natsorted(os.listdir(split_a_path)))

    # Get the set of image file names for each split in directory B
    split_b_images = set(natsorted(os.listdir(split_b_path)))

    # Find the set of images that are in directory A but not in directory B
    only_in_a = split_a_images - split_b_images

    # Find the set of images that are in directory B but not in directory A
    only_in_b = split_b_images - split_a_images

    # Print the results
    print(f"Split: {split}")
    print(f"Only in {dir_a}: {len(only_in_a)}")
    print(f"Only in {dir_b}: {len(only_in_b)}")
    print("-----------")


Split: train
Only in C:\Users\labadmin\PycharmProjects\wirtzlab_pix2pix\datasets\A: 0
Only in C:\Users\labadmin\PycharmProjects\wirtzlab_pix2pix\datasets\B: 0
-----------


## Run this to finish generating dataset: (combine A and B takes quite a bit, but running multiprocessing gives error, todo: find a way to make it work?)

In [1]:
import cv2

In [2]:
!python datasets/combine_A_and_B.py --fold_A C:\Users\labadmin\PycharmProjects\wirtzlab_pix2pix\datasets\pix2pix\fold_A --fold_B C:\Users\labadmin\PycharmProjects\wirtzlab_pix2pix\datasets\pix2pix\fold_B --fold_AB C:\Users\labadmin\PycharmProjects\wirtzlab_pix2pix\datasets\pix2pix\fold_AB

[fold_A] =  C:\Users\labadmin\PycharmProjects\wirtzlab_pix2pix\datasets\pix2pix\fold_A
[fold_B] =  C:\Users\labadmin\PycharmProjects\wirtzlab_pix2pix\datasets\pix2pix\fold_B
[fold_AB] =  C:\Users\labadmin\PycharmProjects\wirtzlab_pix2pix\datasets\pix2pix\fold_AB
[num_imgs] =  1000000
[use_AB] =  True
[no_multiprocessing] =  True
split = test, use 7928/7928 images
split = test, number of images = 7928
split = train, use 63416/63416 images
split = train, number of images = 63416
split = val, use 7927/7927 images
split = val, number of images = 7927


# Pretrained models

Download one of the official pretrained models with:

-   `bash ./scripts/download_pix2pix_model.sh [edges2shoes, sat2map, map2sat, facades_label2photo, and day2night]`

Or add your own pretrained model to `./checkpoints/{NAME}_pretrained/latest_net_G.pt`

In [3]:
# !bash ./scripts/download_pix2pix_model.sh facades_label2photo

In [1]:
import torch
torch.cuda.is_available()

True

# Training

-   `python train.py --dataroot ./datasets/facades --name facades_pix2pix --model pix2pix --direction BtoA`

Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size. Add `--direction BtoA` if you want to train a model to transform from class B to A.

In [ ]:
### 4 hrs per 5 epoch

In [ ]:
!python train.py --dataroot C:\Users\labadmin\PycharmProjects\wirtzlab_pix2pix\datasets\pix2pix\fold_AB --name unstain2stain_pix2pix --model pix2pix --direction AtoB --display_id -1 --display_freq 400 --display_ncols 4 --update_html_freq 1000 --print_freq 100 --checkpoints_dir C:\Users\labadmin\PycharmProjects\wirtzlab_pix2pix\checkpoints --save_epoch_freq 5 --n_epochs 2000

# Testing

-   `python test.py --dataroot ./datasets/facades --direction BtoA --model pix2pix --name facades_pix2pix`

Change the `--dataroot`, `--name`, and `--direction` to be consistent with your trained model's configuration and how you want to transform images.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> Note that we specified --direction BtoA as Facades dataset's A to B direction is photos to labels.

> If you would like to apply a pre-trained model to a collection of input images (rather than image pairs), please use --model test option. See ./scripts/test_single.sh for how to apply a model to Facade label maps (stored in the directory facades/testB).

> See a list of currently available models at ./scripts/download_pix2pix_model.sh

In [ ]:
# !ls checkpoints/

In [ ]:
!python datasets/combine_A_and_B.py --fold_A C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\A --fold_B C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B --fold_AB C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets --no_multiprocessing

In [2]:
!python test.py --dataroot C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets --direction AtoB --model pix2pix --name unstain2stain_pix2pix --results_dir C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\results --num_test 50

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
          

### Inference for proof-of-concept: Tiled original unstained image -> run inference -> Output stained image tiles -> Reconstruct to fake WSI HE Image

### Make A the registered unstained tiles and B the registered HE tiles from the matlab registration code from WSI then run inference

In [21]:
!python test.py --dataroot C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets --model pix2pix --name unstain2stain_pix2pix --results_dir C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\results --direction AtoB --num_test 10995 --load_size 1024 --crop_size 1024

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 1024                          	[default: 256]
                 dataroot: C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: N

# Visualize

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_fake_B.png')
plt.imshow(img)

In [ ]:
img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_real_A.png')
plt.imshow(img)

In [ ]:
img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_real_B.png')
plt.imshow(img)